In [1]:
!git clone https://github.com/ultralytics/yolov5

Cloning into 'yolov5'...
remote: Enumerating objects: 4, done.
remote: Counting objects: 100% (4/4), done.
remote: Compressing objects: 100% (4/4), done.
remote: Total 1798 (delta 0), reused 0 (delta 0), pack-reused 1794
Receiving objects: 100% (1798/1798), 5.07 MiB | 8.00 MiB/s, done.
Resolving deltas: 100% (1171/1171), done.


In [2]:
! pip install -q kaggle

In [3]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle.json


{'kaggle.json': b'{"username":"nahilahmed","key":"744acb2732214a9f195b8f1249866a84"}'}

In [4]:
! mkdir ~/.kaggle

In [5]:
! cp kaggle.json ~/.kaggle/

In [6]:
! chmod 600 ~/.kaggle/kaggle.json

In [7]:
! kaggle datasets list 

ref                                                         title                                             size  lastUpdated          downloadCount  
----------------------------------------------------------  -----------------------------------------------  -----  -------------------  -------------  
roche-data-science-coalition/uncover                        UNCOVER COVID-19 Challenge                       179MB  2020-05-21 18:57:53          16026  
rohanrao/chai-time-data-science                             Chai Time Data Science | CTDS.Show                 3MB  2020-06-20 23:01:09            390  
andrewmvd/data-analyst-jobs                                 Data Analyst Jobs                                  2MB  2020-07-14 08:37:57             53  
mrgeislinger/bart-ridership                                 BART Ridership                                   325MB  2020-07-09 22:28:07              3  
moezabid/zillow-all-homes-data                              Zillow All Homes Data 

In [8]:
%%writefile setup.py

import os

YOLO_DATA = "/content/yolov5/wheat_data"
IMAGES_DATA = YOLO_DATA + "/images"
LABELS_DATA = YOLO_DATA + "/labels"

IMAGE_TRAIN = IMAGES_DATA + "/train"
IMAGE_VAL = IMAGES_DATA + "/validation"
LABEL_TRAIN = LABELS_DATA + "/train"
LABEL_VAL = LABELS_DATA + "/validation"

if not os.path.exists(YOLO_DATA):
  os.mkdir(YOLO_DATA)

if not os.path.exists(IMAGES_DATA):
  os.mkdir(IMAGES_DATA)

if not os.path.exists(LABELS_DATA):
  os.mkdir(LABELS_DATA)

if not os.path.exists(IMAGE_TRAIN):
  os.mkdir(IMAGE_TRAIN)

if not os.path.exists(IMAGE_VAL):
  os.mkdir(IMAGE_VAL)

if not os.path.exists(LABEL_TRAIN):
  os.mkdir(LABEL_TRAIN)

if not os.path.exists(LABEL_VAL):
  os.mkdir(LABEL_VAL)

Writing setup.py


In [ ]:
from tqdm import tqdm
import numpy as np
import shutil

In [ ]:
def process_data(data,data_type="train"):
  for _,row in tqdm(data.iterrows(),total=len(data)):
    image_id = row[0]
    bboxes = row[1]
    yolo_data=[]
    for bbox in bboxes:
      x,y,w,h = bbox 
      x_center = (x + w)/2
      y_center = (y + h)/2

      x_center, y_center, w, h = x_center/1024, y_center/1024, w/1024, h/1024
      yolo_data.append([0,x_center,y_center,w,h])

    yolo_data = np.array(yolo_data)
    np.savetxt(
        os.path.join(YOLO_DATA,"labels/{}/{}.txt".format(data_type,image_id)),
        yolo_data,
        fmt = ["%d", "%f", "%f", "%f", "%f"]
    )
    shutil.copyfile(
        os.path.join(INPUT_PATH,"train/{}.jpg".format(image_id)),
        os.path.join(IMAGES_DATA,"{}/{}.jpg".format(data_type,image_id)),
    )

In [ ]:
process_data(df_train, "train")
process_data(df_valid, "validation")

100%|██████████| 338/338 [00:00<00:00, 420.13it/s]


In [ ]:
!rm global-wheat-detection.zip

In [ ]:
cd yolov5/wheat_data/images/train

/content/yolov5/wheat_data/images/train


In [9]:
%%writefile preparedata.py

import kaggle
import pandas as pd
import os
import ast
from tqdm import tqdm
import numpy as np
import shutil
from sklearn import model_selection
import zipfile


DOWNLOAD_PATH = "/content"
INPUT_PATH = "/content/input"

YOLO_DATA = "/content/yolov5/wheat_data"
IMAGES_DATA = YOLO_DATA + "/images"
LABELS_DATA = YOLO_DATA + "/labels"

def process_data(data,data_type="train"):
  for _,row in tqdm(data.iterrows(),total=len(data)):
    image_id = row[0]
    bboxes = row[1]
    yolo_data=[]
    for bbox in bboxes:
      x,y,w,h = bbox 
      x_center = (x + w)/2
      y_center = (y + h)/2

      x_center, y_center, w, h = x_center/1024, y_center/1024, w/1024, h/1024
      yolo_data.append([0,x_center,y_center,w,h])

    yolo_data = np.array(yolo_data)
    np.savetxt(
        os.path.join(YOLO_DATA,"labels/{}/{}.txt".format(data_type,image_id)),
        yolo_data,
        fmt = ["%d", "%f", "%f", "%f", "%f"]
    )
    shutil.copyfile(
        os.path.join(INPUT_PATH,"train/{}.jpg".format(image_id)),
        os.path.join(IMAGES_DATA,"{}/{}.jpg".format(data_type,image_id)),
    )


kaggle.api.authenticate()
kaggle.api.competition_download_files("global-wheat-detection", path=DOWNLOAD_PATH, quiet=False)

with zipfile.ZipFile(DOWNLOAD_PATH + "/global-wheat-detection.zip", 'r') as zip_ref:
    zip_ref.extractall(INPUT_PATH)


if __name__ == "__main__":
  df = pd.read_csv(os.path.join(INPUT_PATH,"train.csv"))
  df.bbox = df.bbox.apply(ast.literal_eval)
  df = df.groupby("image_id")["bbox"].apply(list).reset_index(name="bboxes")
  print(df.head(10))

  df_train, df_valid = model_selection.train_test_split(
      df,
      test_size=0.1,
      shuffle=True,
      random_state = 42
  )

  df_train = df_train.reset_index(drop=True)
  df_valid = df_valid.reset_index(drop=True)

  process_data(df_train, "train")
  process_data(df_valid, "validation")

Writing preparedata.py


In [10]:
! python setup.py

In [11]:
! python preparedata.py

 95% 579M/607M [00:12<00:01, 27.4MB/s]
100% 607M/607M [00:12<00:00, 51.8MB/s]
    image_id                                             bboxes
0  00333207f  [[0, 654, 37, 111], [0, 817, 135, 98], [0, 192...
1  005b0d8bb  [[765.0, 879.0, 116.0, 79.0], [84.0, 539.0, 15...
2  006a994f7  [[437.0, 988.0, 98.0, 36.0], [309.0, 527.0, 11...
3  00764ad5d  [[89.0, 256.0, 113.0, 107.0], [216.0, 282.0, 1...
4  00b5fefed  [[709.0, 97.0, 204.0, 105.0], [775.0, 250.0, 1...
5  00b70a919  [[147.0, 46.0, 80.0, 88.0], [0.0, 230.0, 69.0,...
6  00e903abe  [[0, 103, 22, 72], [0, 8, 76, 90], [59, 64, 61...
7  00ea5e5ee  [[327.0, 3.0, 35.0, 49.0], [928.0, 0.0, 90.0, ...
8  010b216d4  [[367.0, 293.0, 113.0, 110.0], [0.0, 556.0, 91...
9  010c93b99  [[185.0, 559.0, 146.0, 113.0], [691.0, 809.0, ...
100% 3035/3035 [00:04<00:00, 690.41it/s]
100% 338/338 [00:00<00:00, 737.66it/s]


In [12]:
cd yolov5/

/content/yolov5


In [13]:
%%writefile wheat.yaml

train: wheat_data/images/train/
val: wheat_data/images/validation/
nc: 1
names: ["wheat"]

Writing wheat.yaml


In [14]:
!cat wheat.yaml


train: wheat_data/images/train/
val: wheat_data/images/validation/
nc: 1
names: ["wheat"]

In [15]:
import torch
torch.cuda.is_available()

True

In [ ]:
cd ..

/content/yolov5


In [16]:
!pip install -qr requirements.txt  # install dependencies (ignore errors)

     |████████████████████████████████| 276kB 7.0MB/s 


In [17]:
! python train.py --img 1024 --batch 8 --epochs 10 --data wheat.yaml --cfg models/yolov5s.yaml --name wm --weights yolov5s.pt --device 0

Apex recommended for faster mixed precision training: https://github.com/NVIDIA/apex
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

Namespace(batch_size=8, bucket='', cache_images=False, cfg='models/yolov5s.yaml', data='wheat.yaml', device='0', epochs=10, evolve=False, hyp='', img_size=[1024, 1024], local_rank=-1, multi_scale=False, name='wm', noautoanchor=False, nosave=False, notest=False, rect=False, resume=False, single_cls=False, sync_bn=False, total_batch_size=8, weights='yolov5s.pt', world_size=1)
Start Tensorboard with "tensorboard --logdir=runs", view at http://localhost:6006/
2020-07-21 05:50:30.988160: I tensorflow/stream_executor/platform/default/dso_loader.cc:44] Successfully opened dynamic library libcudart.so.10.1
Hyperparameters {'optimizer': 'SGD', 'lr0': 0.01, 'momentum': 0.937, 'weight_decay': 0.0005, 'giou': 0.05, 'cls': 0.58, 'cls_pw': 1.0, 'obj': 1.0, 'obj_pw': 1.0, 'iou_t': 0.2, 'anchor_t': 4.0, 'fl_gamma': 0.0, 'hsv_h': 0.014, '

In [36]:
! python detect.py --source /content/input/test --weights /content/yolov5/runs/exp0_wm/weights/best_wm.pt --conf 0.4

Namespace(agnostic_nms=False, augment=False, classes=None, conf_thres=0.4, device='', img_size=640, iou_thres=0.5, output='inference/output', save_txt=False, source='/content/input/test', update=False, view_img=False, weights=['/content/yolov5/runs/exp0_wm/weights/best_wm.pt'])
Using CUDA device0 _CudaDeviceProperties(name='Tesla K80', total_memory=11441MB)

Fusing layers... Model Summary: 140 layers, 7.24652e+06 parameters, 6.61683e+06 gradients
image 1/10 /content/input/test/2fd875eaa.jpg: 640x640 Done. (0.034s)
image 2/10 /content/input/test/348a992bb.jpg: 640x640 Done. (0.033s)
image 3/10 /content/input/test/51b3e36ab.jpg: 640x640 Done. (0.034s)
image 4/10 /content/input/test/51f1be19e.jpg: 640x640 Done. (0.033s)
image 5/10 /content/input/test/53f253011.jpg: 640x640 Done. (0.033s)
image 6/10 /content/input/test/796707dd7.jpg: 640x640 Done. (0.033s)
image 7/10 /content/input/test/aac893a91.jpg: 640x640 Done. (0.033s)
image 8/10 /content/input/test/cb8d261a3.jpg: 640x640 Done. (0.033